In [2]:
import warnings
warnings.filterwarnings('ignore')

import math
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler

In [11]:
dolar_df = pd.read_csv('./data-projects/dolar_D_2307.csv')
dolar_df.set_index('Data', inplace=True)
dolar_df

,Abertura,Máxima,Mínima,Fechamento,BOP,VOL,Compradores,Vendedores,Saldo
Data,,,,,,,,,
2021-07-23,5203.5000,5236.5000,5163.0000,5203.5000,0.0000,5.767838e+10,95580,-97285,-1705
2021-07-22,5186.5000,5229.5000,5173.5000,5205.5000,0.3393,4.981865e+10,86065,-84660,1405
2021-07-21,5238.0000,5283.5000,5187.0000,5194.5000,-0.4508,8.043548e+10,119365,-129670,-10305
2021-07-20,5250.0000,5301.0000,5208.5000,5228.0000,-0.2378,7.046589e+10,110000,-117505,-7505
2021-07-19,5168.5000,5265.5000,5159.5000,5259.0000,0.8538,7.681582e+10,128115,-123845,4270
...,...,...,...,...,...,...,...,...,...
2014-03-11,3685.3853,3705.6047,3659.7221,3698.6057,0.2881,6.058720e+10,0,0,0
2014-03-10,3664.3881,3687.7183,3641.8357,3678.3862,0.3051,5.786549e+10,0,0,0
2014-03-07,3635.6143,3678.3862,3633.2813,3664.3881,0.6379,5.799550e+10,0,0,0
